# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [3]:
pd_data['age'].nunique()

3

In [4]:
pd_data['income'].nunique()

3

## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.
->한쪽으로 몰려 노드를 최소화하는 것이 좋은 모델이기에 gini index를 감소시키는 것이 목적이다.

In [5]:
def get_gini(df, label):
    gini = 1
    percentage = df[label].value_counts() / len(df) #
    #전체 데이터 값에서 클래스의 데이터 수
    for p in percentage:
        gini -= p**2
        
    return gini

In [6]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

In [7]:
get_gini(pd_data,'income')

0.653061224489796

In [8]:
get_gini(pd_data,'age')

0.663265306122449

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [9]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    result=[] #빈 리스트 생성
    len_uniques = len(uniques) #nunique와 같은 결과 생성
    for i in range(len_uniques - 1):
        for j in list(combinations(uniques, i+1)): #{A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B}) 생성
            result.append(list(j)) #하나의 ({A}, {B,C})만들 때마다 비어있는 result list에 추가
    
    return result

In [10]:
#변수의 class 3개 income
get_binary_split(pd_data, "income") #6개의 경우의 수 완성

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [11]:
#변수의 class 3개 age
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [12]:
def get_attribute_gini_index(df, attribute, label): 
    
    result = {} #빈 리스트 생성
    binary_split = get_binary_split(df, attribute) #함수 get_binary_split의 결과를 binary split에 담음.
    n = len(df) #데이터의 개수
    for i in binary_split: #모든 경우의 수를 돌아가면서
        key = '_'.join(i) 
        #"(구분자)".join(list)-매게변수로 들어온 리스트에 있는 요소들을 합쳐 하나의 문자열로 바꾸어 변환하는 함수
        #ex)매개변수 ['a','b','c']->abc
        #이 경우 binary split의 경우의 수들이 -를 중심으로 이어져 문자열로 나옴.
        df_1 = df[df[attribute].isin(i)] # binary split 중 i를 포함한 경우
        df_2 = df[~df[attribute].isin(i)] # binary split 중 i를 포함하지 않은 경우 (i 외의경우)
        gini_index = len(df_1) / n * get_gini(df_1, label) + len(df_2) / n * get_gini(df_2, label)
        result[key] = gini_index
    
    return result

In [13]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'high_low': 0.4583333333333333,
 'high_medium': 0.45,
 'low': 0.45,
 'medium': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [14]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())#제일 작은 지니계수를 도출하는 class 명과 해당 gini index

('high', 0.4428571428571429)

In [15]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0] #제일 작은 지니계수를 도출하는 class 명만

'high'

age의 middel_aged가 가장 낮은 gini index를 보인다.

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [16]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:

    min_val = min(get_attribute_gini_index(pd_data, feature, 'class_buys_computer').items())
    print("Minimum Gini index of " + feature +" : " + str(round(min_val[1], 4)))

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [17]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'middle_aged': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937,
 'senior': 0.4571428571428572,
 'youth': 0.3936507936507937,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [18]:
#선정한 feature로 데이터프레임 split을 해주세요.

df_1 = pd_data[pd_data['age'] == 'middle_aged']#middle_aged의 데이터 프레임
df_2 = pd_data[pd_data['age'] != 'middle_aged'] #middle_aged 외의 데이터 프레임

In [19]:
df_1.head() #split 결과 확인 

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [20]:
df_2.head() #split 결과 확인 

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [21]:
features = [x for x in list(pd_data.columns)  if x != 'age']  # 탐색할 피쳐들 (income 제외)
# res = pd.DataFrame(columns=['feature', 'attribute', 'gini'])
# res.loc['middle_aged'] = propose_feature(pos, left_features)
# res.loc['not middle_aged'] = propose_feature(neg, left_features)
# res
print(list(pd_data.columns))

['age', 'income', 'student', 'credit_rating', 'class_buys_computer']


In [22]:
# 각 변수를 대상으로 반복문 수행
# 결과는 소수점 이하 4자리까지 출력

#출력문 확인하니 min_class에 들어갈 class는 income,student,credit_rating 3가지임
ft = ["income","student","credit_rating"]

for feature in ft:
  print("Age : Middle_aged,", "Minimum Gini index of", ft, ":", round(min(get_attribute_gini_index(df_1, feature, label).values()), 4))
#middle_aged에서 가장작은 gini index를 구해준 후,round함수를 이용해 소수점 넷째자리까지 잘라줌,

print("################################################################")

for feature in ft:
      print("Age : youth_senior ,", "Minimum Gini index of", feature, ":", round(min(get_attribute_gini_index(df_2, feature, label).values()), 4))
 


Age : Middle_aged, Minimum Gini index of ['income', 'student', 'credit_rating'] : 0.0
Age : Middle_aged, Minimum Gini index of ['income', 'student', 'credit_rating'] : 0.0
Age : Middle_aged, Minimum Gini index of ['income', 'student', 'credit_rating'] : 0.0
################################################################
Age : youth_senior , Minimum Gini index of income : 0.375
Age : youth_senior , Minimum Gini index of student : 0.32
Age : youth_senior , Minimum Gini index of credit_rating : 0.4167


## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

Entropy란?
불확실성의 척도

In [23]:
from math import log2

def getEntropy(df, feature) :
    
    entropy = 0 #entropy 초기화
    unique = list(df[feature].unique())  #해당 feature에서 고유한(겹치지 않는 값의 리스트 생성)
  
    for i in unique: 
        p=sum(df[feature]==i)/len(df[feature]) #각 고유한 feature을 가지는 데이터의 개수/전체 데이터의 개수
        entropy += - p*log2(p) #각 고유한 feature의 함수값을 더해주면 entropy가 나온다.
    return entropy


In [24]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706309

In [25]:
def getGainA(df, feature) :
        
    result = {} #빈 리스트 생성

    info_D = getEntropy(df, feature) # 목표변수에 대한 Entropy 를 구하기
    columns = list(df.loc[:, df.columns != feature]) # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장
   
    for i in columns: #각 칼럼별 엔트로피를 구해줘야 하니 돌림.
      info_cl=0
      unique = list(df[i].unique()) #column에 존재하는 고유한 값들
      for j in unique:
         f_l = len(df[feature])     # f_l : 전체 데이터 개수
         cf_l = sum(df[i] == j)     # cf_l : 전체 데이터 중 i 컬럼의 값이 j인 데이터의 개수 (= 가중치)
         info_cl += (f_l / cf_l) * getEntropy(df[df[i] == j], feature) 
      result[i] = info_D-info_cl

    return result

In [26]:
getGainA(pd_data, "class_buys_computer")

{'age': -4.497037370275512,
 'credit_rating': -2.8127840924661847,
 'income': -7.5418777563968105,
 'student': -2.213515870562527}